# Problems

1. In order to produce plots/paths, we need to run simulations, which means that we need to close loops. This means model capture.
2. In order to close loops, we need to allow for non-differentiable functions. This means a single step may involve solving a differential equation, or an optmisation problem.
3. In order to perform co-design (multi-domain multi-parameter optimisation) we need to evaluate the change a parameter has upon a loss function, which may require expensive computes to be replaced by a surrogate

# Required Information
## Codesign Problems
### One-shot Codesign Problem
Required Inputs:
- A loss function (what makes a good path good?)
- A reference trajectory, or some other specification.
- Models (dynamics, IO & constraints).
- Tunable parameters.
- value and integral constraints.

Required outputs:
- Loss as a function of parameters, and the 'best' parameter set found.

### Operational Codesign problem
Required Inputs:
- A set (or sampler) of reference paths, corresponding loss functions and weights (if using single objective)
- Value and integral constraints
- Models
- Tunable parameters

Required Output:
- Loss distribution over sampled paths for the 'best' parameter set.

## Challenges
Challenge:
- Nonlinear optimisation
- Some gradients are more expensive to evaluate than others

_as models get more realistic, they get more expensive, noisier and less observable._


# Required Infrastructure
- Stiff Ode solver / multi-scale solver
- Noise

Require 3 update loops:
    - non-fixed time-step Implicit or Explicit continuous systems (analogue signals)
    - fixed time step updates (discrete sequences)
    - parameter updates



In [ ]:
from numpy import sin, cos
Beta = [-3.7693 * 1e5,
        -3.7225 * 1e4,
         2.6814 * 1e4,
        -1.7277 * 1e4,
         3.5542 * 1e4,
        -2.4216 * 1e3,
        6.3785 * 1e3,
        -1.0090 * 1e2
]
C_L = [4.6773, -1.8714 * 1e-2]
C_D = [5.8224, -4.5315*1e-2, 1.0131 *1e-2]
C_Ma = [6.2926, 2.1335]
C_Md = [-1.2897,1.8979 * 1e-1]
g = 32.174
m = 3 * 1e2
I_yy = 5 * 1e5
rho = 6.7429 *1e-5
S = 17
c_bar = 17
z_t = 8.36
h_0 = 8.5 *1e4
h_s = 2.1358 * 1e4
zeta = 0.7
omega = 20
import sympy as sp

def lie_derivative(f, X, basis, order=0):

    assert len(X) == len(basis), "Vector field must be of the same size as the basis"
    if order > 0:
        f_prime = lie_derivative(f, X, basis, order - 1)
        df = (f_prime(basis).diff(x) for x in basis)
    else:
        df = (f(basis).diff(x) for x in basis)
    return sum(X_i * df_i for X_i, df_i in zip(X, df))

def aerodynamic_coeff(x, u):
    h, V, alpha,theta, Q, phi,dphi = x
    delta_e, _ = u
    # First term when i = 0 corresponds to beta_1 in Vincent's report
    # the contribution to the sum is beta_1 * phi * alpha ^ 3
    # when i = 3, the contribution is beta_4 * alpha ^ 2
    # when i = 7 the contribution is beta_8
    # so, odd orders get a phi coefficient
    # alpha power descend from 3 in pairs
    T = sum((beta * phi if i % 2 == 0 else beta) * alpha ** (3 - i//2) for i, beta in enumerate(Beta))
    c = 0.5 * S * (rho * V ** 2)
    L = c * C_L[0] * alpha + C_L[0]
    D = c * sum(cd * alpha ** (2 - i) for i, cd in enumerate(C_D))
    M = z_t * T + c * c_bar * (
            sum(ca * alpha** (2 - i) for i, ca in enumerate(C_Ma))
            + sum(cd * delta_e ** (1 - i) for i, cd in enumerate(C_Md))
    )

    return T, L, D, M

x0 = [85000, 7702.08, 3.682, 3.682, 0, 0.16176, 0]
u0 = [16.3618, 0.16176]

def dxdt(t, x, xdot, u):
    # this is affine in control u

    h, V, alpha,theta, Q, phi, dphi = x
    _, phi_c = u
    T, L, D, M = aerodynamic_coeff(x, u)
    gamma  =  theta - alpha
    xdot[0] = V * sin(gamma)
    xdot[1] = (T*cos(alpha) - D) / m - g * sin(gamma)
    xdot[2] = -(T*sin(alpha) + L)/ (m * V) + Q + g*cos(gamma) / V
    xdot[3] = Q
    xdot[4] = M/I_yy

    # phi
    xdot[5] = dphi
    xdot[6] = -2 * zeta * omega * dphi + (phi_c - phi)* omega**2

# TODO: Implement Vincents reference governer


def solve():
    # pick a timespan and a sampling rate for the discrete system (eg 100Hz)
    # each t_{k-1} -> t_{k}, solve using an appropriate solver
    # update all discrete systems in order of dependency (if resolvable)
    # repeat
    pass



# Procedure

### Step 1.
Step 1 should be to algorithmically non-dimensionalize the problem.
Assume we're linearising about the origin (otherwise, perform change of variables until that is the case)
compute the Normal Form (either Schur or Jordan) of $DF$ such that $J DF_0 J^{-1} = \Lambda + A$, then change $x' = J^{-1}x$ where $\Lambda$ is a diagonal matrix of eigenvalues and $A$ is upper triangular

the condition factor $\kappa$ dictates the time scales at play... If it $log_{10}\kappa > 2$, then we probably want to treat it as a multiscale system.

let $x' = Sx$, where $S$ is a diagonal matrix.
minimize $sum_i ||S^T D^2[F_i]_{x=0} S|| + eps*||S - I||_1$

